In [ ]:
from dataclasses import dataclass

@dataclass
class Product:
    sku: str
    warehouse: str
    node_type: str
    current_inventory: int
    detailed_incoming_inventory: dict
    forecast: dict
    forecast_error_model: dict
    days_to_next_review: int
    transfer_unit: int
    desired_service_level: float
    is_superwh: bool
    #remaining_shelf_life_units: dict = default_factory(dict)


@dataclass
class Supplier:
    external_id: str
    lead_time_model: dict
    delay_model: dict = None
    



In [ ]:
origin_product = Product(
                sku="123",
                warehouse="VLP",
                node_type="offering",
                current_inventory=4500,
                detailed_incoming_inventory={
                    2030:{
                        "2024-09-30": 10,
                        "2021-10-02": 20,
                        "2021-10-10": 30
                    }
                },
                forecast={'2024-09-26': 128.0, '2024-09-27': 107.0, '2024-09-28': 56.0, '2024-09-29': 0.0, '2024-09-30': 119.0, '2024-10-01': 111.0, '2024-10-02': 119.0, '2024-10-03': 103.0, '2024-10-04': 91.0, '2024-10-05': 89.0, '2024-10-06': 0.0, '2024-10-07': 131.0, '2024-10-08': 111.0, '2024-10-09': 120.0, '2024-10-10': 103.0},
                forecast_error_model={
                    'distribution': 'NORM',
                    'mu':0.0,
                    'sigma': 35
                },
                days_to_next_review=7,
                transfer_unit=50,
                desired_service_level=0.9
            )

destination_product = Product(
                sku="123",
                warehouse="VLP",
                node_type="demand",
                current_inventory=500,
                detailed_incoming_inventory={
                    2030:{
                        "2021-10-10": 30
                    }
                },
                forecast={'2024-09-26': 128.0, '2024-09-27': 127.0, '2024-09-28': 96.0, '2024-09-29': 0.0, '2024-09-30': 169.0, '2024-10-01': 91.0, '2024-10-02': 129.0, '2024-10-03': 130.0, '2024-10-04': 111.0, '2024-10-05': 189.0, '2024-10-06': 0.0, '2024-10-07': 131.0, '2024-10-08': 111.0, '2024-10-09': 120.0, '2024-10-10': 103.0},
                forecast_error_model={
                    'distribution': 'NORM',
                    'mu':0.0,
                    'sigma': 45
                },
                days_to_next_review=7,
                transfer_unit=50,
                desired_service_level=0.9
            )



test_supplier = Supplier(
    external_id="abc",
    lead_time_model={
        7:0.5,
        8:0.3,
        9:0.2
    },
    delay_model={0:1}
)

transfers_supplier = Supplier(
    external_id="Frubana",
    lead_time_model={
        1:1
    },
    delay_model={0:1}
)



In [ ]:
from datetime import datetime, timedelta
import numpy as np
# random truncted normal distribution
import scipy.stats


def generate_truncated_normal(mu, sigma, lower_bound, upper_bound=np.infty, sample_size=100000):
    lower_bound = (lower_bound - mu) / sigma
    upper_bound = (upper_bound - mu) / sigma
    return np.round(scipy.stats.truncnorm.rvs(lower_bound, upper_bound, loc=mu, scale=sigma, size=sample_size), 0)

def generate_demand_scenario(demand_distribution: dict, sample_size: int = 100000):
    mu = demand_distribution.get("mu", 0)
    sigma = demand_distribution.get("sigma", 0.0001)
    if demand_distribution.get("distribution", "NORM") == "NORM":
        return np.random.normal(mu, sigma, sample_size)
    else:
        raise NotImplementedError

def generate_lead_time_scenario(lead_time_distribution: dict, days_to_next_review, sample_size: int = 100000):
   return np.random.choice(list(lead_time_distribution.keys()), sample_size, p=list(lead_time_distribution.values()))+days_to_next_review



In [ ]:
def simulate_origin(producto, supplier, sample_size=10000):
    
    lt = generate_lead_time_scenario(supplier.lead_time_model,producto.days_to_next_review,  sample_size ) 

    simulation_dates=[datetime.strftime(datetime.strptime(min(producto.forecast.keys()), "%Y-%m-%d")  + timedelta(days=i), "%Y-%m-%d") for i in range(max(lt), )]
    planning_horizon_length = len(simulation_dates)


    waste = np.zeros((sample_size, planning_horizon_length))
    lost_sales = np.zeros((sample_size, planning_horizon_length))
    stockouts = np.zeros((sample_size, planning_horizon_length))


    Q_transfer = 0
    results_by_transfer = {}

    while Q_transfer < producto.current_inventory:
        inventory = np.zeros(sample_size) + producto.current_inventory - Q_transfer

        trunc_demand_scenarios = np.array([generate_truncated_normal(producto.forecast.get(date, 0)+ producto.forecast_error_model["mu"], producto.forecast_error_model["sigma"], 0, np.infty, sample_size)*(datetime.strptime(date, "%Y-%m-%d").weekday() !=6) for i, date in enumerate(simulation_dates)]).T

        # for each row make zero all the columns that are beyond the lead time
       
                                                                                                                                                                                                          

        for i, date in enumerate(simulation_dates):
            if date in producto.forecast:
                demand = trunc_demand_scenarios[:, i]*(i<=lt)
                lost_sales[:, i] = np.maximum(0, demand - inventory)
                inventory = np.maximum(0, inventory - demand)+producto.detailed_incoming_inventory.get(2030, {}).get(date, 0)
                stockouts[:, i] = inventory <= 0
        new_res = {
            "lost_sales": np.mean(np.sum(lost_sales[:, :-1], axis=1)),
            "stockouts": np.mean(np.sum(stockouts[:, :-1], axis=1) > 0),
            "inventory": np.mean(inventory)
        }
        if new_res['stockouts'] >1- producto.desired_service_level:
            break
        results_by_transfer[Q_transfer] = new_res

        Q_transfer += producto.transfer_unit
    return results_by_transfer

def simulate_destination(producto, supplier, sample_size=10000):
    
    lt = generate_lead_time_scenario(supplier.lead_time_model,producto.days_to_next_review,  sample_size ) 

    simulation_dates=[datetime.strftime(datetime.strptime(min(producto.forecast.keys()), "%Y-%m-%d")  + timedelta(days=i), "%Y-%m-%d") for i in range(max(lt), )]
    planning_horizon_length = len(simulation_dates)


    lost_sales = np.zeros((sample_size, planning_horizon_length))
    stockouts = np.zeros((sample_size, planning_horizon_length))

    Q_transfer = 0
    results_by_transfer = {}

    while True:
        inventory = np.zeros(sample_size) + producto.current_inventory + Q_transfer

        trunc_demand_scenarios = np.array([generate_truncated_normal(producto.forecast.get(date, 0)+ producto.forecast_error_model["mu"], producto.forecast_error_model["sigma"], 0, np.infty, sample_size)*(datetime.strptime(date, "%Y-%m-%d").weekday() !=6) for i, date in enumerate(simulation_dates)]).T

        # for each row make zero all the columns that are beyond the lead time
       
                                                                                                                                                                                                          

        for i, date in enumerate(simulation_dates):
            if date in producto.forecast:
                demand = trunc_demand_scenarios[:, i]*(i<=lt)
                lost_sales[:, i] = np.maximum(0, demand - inventory)
                inventory = np.maximum(0, inventory - demand)+producto.detailed_incoming_inventory.get(2030, {}).get(date, 0)
                stockouts[:, i] = inventory <= 0
        new_res = {
            "lost_sales": np.mean(np.sum(lost_sales[:, :-1], axis=1)),
            "stockouts": np.mean(np.sum(stockouts[:, :-1], axis=1) > 0),
            "inventory": np.mean(inventory)
        }

        results_by_transfer[Q_transfer] = new_res

        if new_res['stockouts'] <1- producto.desired_service_level:
            break
        Q_transfer += producto.transfer_unit
    return results_by_transfer

print(simulate_origin(origin_product, test_supplier))


In [ ]:
print(simulate_destination(destination_product, transfers_supplier))


In [ ]:
trunc_demand_scenarios = np.array([generate_truncated_normal(100+
    test_product.forecast_error_model["mu"], test_product.forecast_error_model["sigma"], 0, np.infty, sample_size) for _ in range(planning_horizon_length)]).T

total_demand = np.sum(trunc_demand_scenarios, axis=1)
total_demand

from matplotlib import pyplot as plt
plt.hist(total_demand, bins=100)

R=np.percentile(total_demand, 75)

excess = np.maximum(0, total_demand-R)
np.mean(excess)